# HW1_DecisionTree

## Importing libaries

In [29]:
import csv
import numpy as np
import pandas as pd
import random
from pprint import pprint
import yaml

## Load Dataset

In [30]:
df = pd.read_csv("nursery.csv")                         # importing dataset with pandas
df = df.rename(columns={"final evaluation": "label"})   # renaming last column as "label" 

In [31]:
header = df.columns                                     # collecting all the features and target

In [32]:
df.head()                                               # showing the first 5 rows of dataset

,parents,has_nurs,form,children,housing,finance,social,health,label
0,usual,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
1,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
2,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
3,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
4,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority


In [33]:
df.info()                                              # more information about the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12960 entries, 0 to 12959
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   parents   12960 non-null  object
 1   has_nurs  12960 non-null  object
 2   form      12960 non-null  object
 3   children  12960 non-null  object
 4   housing   12960 non-null  object
 5   finance   12960 non-null  object
 6   social    12960 non-null  object
 7   health    12960 non-null  object
 8   label     12960 non-null  object
dtypes: object(9)
memory usage: 911.4+ KB


In [34]:
df.describe()                                          # more information about the dataset

,parents,has_nurs,form,children,housing,finance,social,health,label
count,12960,12960,12960,12960,12960,12960,12960,12960,12960
unique,3,5,4,4,3,2,3,3,5
top,usual,proper,complete,1,convenient,convenient,nonprob,recommended,not_recom
freq,4320,2592,3240,3240,4320,6480,4320,4320,4320


## Split Dataset to train and test

In [35]:
def train_test_split(df, test_size):
    """ Splits the dataset(df) to train and test with the number/pertentage of test dataset. """
    
    if isinstance(test_size, float):                                   # if test size is proportion (%)
        test_size = round(test_size * len(df))

    indices = df.index.tolist()                                        # all the indexes
    test_indices = random.sample(population=indices, k=test_size)      # choosing randomly the indicies for test

    test_df = df.loc[test_indices]                                     # collecting the test dataset        
    train_df = df.drop(test_indices)                                   # drop test dataset for making train dataset
    
    return train_df, test_df

In [36]:
random.seed(0)                                              # initialize the random number generator
train_df, test_df = train_test_split(df, test_size=0.25)    # collect 20% of dataset for test

In [37]:
data = train_df.to_numpy()                                  # tranforming dataset to numpy array

## Entropy & Gini Functions

$$E(S)=\sum_{i=1}^{c} −p_i log_2p_i$$

In [38]:
def entropy(labels):
    """ Computes entropy of label distribution. """

    n_labels = len(labels)                                   # number of data 

    if n_labels <= 1:
        return 0

    value, counts = np.unique(labels, return_counts=True)    # unique classes & number of data in label
    probs = counts / n_labels                                # probability of each class
    n_classes = np.count_nonzero(probs)                      # number of classes in label   

    if n_classes <= 1:                                      
        return 0

    ent = 0.

    # Compute entropy
    for i in probs:                                         # compute entropy using  
        ent -= i * np.log2(i)                               # probabilities of each class and log2

    return ent

$$Gini=1 - \sum_{i=1}^{c} (p_i)^2$$

In [39]:
def gini(labels):
    """
    Given the Series, it calculates the Gini Impurity. 
    y: variable with which calculate Gini Impurity.
    """
    unique, counts = np.unique(labels, return_counts=True)  # unique classes & number of data in label
    p = counts / labels.shape[0]                            # probability of each class
    gini = 1 - np.sum(p**2)                                 # calculate gini for each class using formula
    
    return (gini)

## Split the data at the node into left and right branches

In [40]:
def split_dataset(X, node_indices, feature):
    """
    Splits the data at the given node into
    left and right branches
    
    Args:
        X (ndarray):             Data matrix of shape(n_samples, n_features)
        node_indices (ndarray):  List containing the active indices. I.e, the samples being considered at this step.
        feature (int):           Index of feature to split on
    
    Returns:
        result (dict): Indices with feature value
    """
    result = dict()
    for value in set(X[:, feature]):            # building the output dictionary for each feature
        result[str(value)] = []

    for i in node_indices:                      # appending the active indices to the dictionary
        result[X[i, feature]].append(i)

    return result

## Information gain of the node

This metric indicates the improvement when making different partitions and is usually used with entropy (it could also be used with the Gini index, although in that case it would not be called Informaiton Gain).

In [41]:
def compute_information_gain(X, y, node_indices, feature, impurity_func=gini):
    
    """
    Compute the information of splitting the node on a given feature
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
   
    Returns:
        cost (float):        Cost computed
    
    """    

    # Split dataset
    splited = split_dataset(X, node_indices, feature)
    
    # Some useful variables
    X_node, y_node = X[node_indices], y[node_indices]
    n = len(y_node)  
    
    if n == 0:
        return 0
    
    information_gain = 0                                                    # initializing variables
    conditional_impurity = 0
    
    for key, value in splited.items():                                      # calculating conditional entropy for each class
        conditional_impurity += (impurity_func(y[value]) * (len(value) / n))
    
    information_gain = impurity_func(y_node) - conditional_impurity          # calculating information gain using formula
    
    return information_gain

## Best feature to split the node data 

In [42]:
def get_best_split(X, y, node_indices, used_feature, impurity_func):   
    """
    Returns the optimal feature and threshold value
    to split the node data 
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.

    Returns:
        best_feature (int):     The index of the best feature to split
    """    
    
    # Some useful variables
    num_features = X.shape[1]
    best_feature = -1                                                               # initializing variables
    max_info_gain = 0
    
    for feature in range(num_features):                                             # finding the maximum information gain
        if feature in used_feature:
            continue
        info_gain = compute_information_gain(X, y, node_indices, feature=feature, impurity_func=impurity_func)
        if (info_gain >= max_info_gain):
            max_info_gain = info_gain
            best_feature = feature    
   
    return best_feature

## Tree algorithm

In [43]:
def build_tree_recursive(data, node_indices, used_feature, branch_name, max_depth, current_depth, impurity_func=entropy):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree. 
        current_depth (int):    Current depth. Parameter used during recursive call.
   
    """ 
    X = data[:, :-1]                            # dataset 
    y = data[:, -1]                             # target
    
    formatting = "-"*current_depth
    
    # if y had only one class 
    if len(set(y[node_indices])) == 1:
        return y[node_indices[0]]
    
    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        value, counts = np.unique(y[node_indices], return_counts=True)
        formatting = " "*current_depth + "-"*current_depth
        if len(counts) == 0:
            return
        return value[np.argmax(counts)]
    
    

    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices, used_feature, impurity_func) 
    used_feature.append(best_feature)
    formatting = "-"*current_depth
#     print("%s Depth %d, %s: Split on feature: %s" % (formatting, current_depth, branch_name, header[best_feature]))
    
    # Split the dataset at the best feature
    splited = split_dataset(X, node_indices, best_feature)
    
    feature_name = header[best_feature]
    question = "{}".format(feature_name)

    # instantiate sub-tree
    sub_tree = {question: []}

    for feature, indices in splited.items():
        used_feature_copy = used_feature.copy()
        branch = build_tree_recursive(data, indices, used_feature_copy, feature, max_depth, current_depth+1)
        sub_tree[question].append({feature: branch})

    return sub_tree

## Accuracy

In [44]:
def calculate_accuracy(df, tree):
    af = {}
    af["classification"] = df.apply(classify_example, axis=1, args=(tree,))
    af["classification_correct"] = af["classification"] == df["label"]
    accuracy = af["classification_correct"].mean()
    
    return accuracy

## Classify an example using the tree

In [45]:
def classify_example(example, tree):
    
    question = list(tree.keys())[0]
    values = tree[question]

    i = 0
    for value in values:
        name_of_value = list(value.keys())[0]
        
        if str(example[question]) == name_of_value:
            answer = list(tree[question][i].values())[0]
            
        i += 1

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

## Assemble

In [46]:
def train(df, train_size, max_depth, impurity_func):
    train_df, test_df = train_test_split(df, test_size=1-train_size)
    data = train_df.to_numpy() 
    root_indices = range(data.shape[0])
    used_feature = []
    tree = build_tree_recursive(data, root_indices, used_feature, "Root", max_depth=max_depth, current_depth=0, impurity_func=impurity_func)
    return train_df, test_df, tree

In [47]:
def question(df, train_size, max_depth):
    train_df, test_df, tree = train(df, train_size, max_depth, entropy)
    with open(f'results/result{train_size*100}-{max_depth}-entrpoy.yml', 'w') as f:
        yaml.dump(tree, f)
    print("Entropy")
    print(f"Accuracy on total: {calculate_accuracy(df, tree)}")
    print(f"Accuracy on train: {calculate_accuracy(train_df, tree)}")
    print(f"Accuracy on test: {calculate_accuracy(test_df, tree)}")
    
    train_df, test_df, tree = train(df, train_size, max_depth, gini)
    with open(f'results/result{train_size*100}-{max_depth}-gini.yml', 'w') as f:
        yaml.dump(tree, f)
    print("Gini")
    print(f"Accuracy on total: {calculate_accuracy(df, tree)}")
    print(f"Accuracy on train: {calculate_accuracy(train_df, tree)}")
    print(f"Accuracy on test: {calculate_accuracy(test_df, tree)}")

# Use model

## a) Train: 80%

In [66]:
train_df, test_df, tree = train(df, 0.8, 3, gini)

In [67]:
pprint(tree, width=15)

{'health': [{'priority': {'has_nurs': [{'critical': {'parents': [{'pretentious': 'spec_prior'},
                                                                 {'great_pret': 'spec_prior'},
                                                                 {'usual': 'spec_prior'}]}},
                                       {'less_proper': {'parents': [{'pretentious': 'priority'},
                                                                    {'great_pret': 'spec_prior'},
                                                                    {'usual': 'priority'}]}},
                                       {'improper': {'parents': [{'pretentious': 'spec_prior'},
                                                                 {'great_pret': 'spec_prior'},
                                                                 {'usual': 'priority'}]}},
                                       {'very_crit': {'children': [{'2': 'spec_prior'},
                                                         

In [68]:
print(f"Accuracy: {calculate_accuracy(train_df, tree)}")

Accuracy: 0.8915895061728395


In [69]:
example = test_df.iloc[0]
example

parents             usual
has_nurs         critical
form               foster
children                2
housing          critical
finance            inconv
social      slightly_prob
health          not_recom
label           not_recom
Name: 3344, dtype: object

In [70]:
classify_example(example, tree)

'not_recom'

## b-1) Train: 75%, max_depth = 6

In [52]:
question(df, train_size=0.75, max_depth=6)

Entropy
Accuracy on total: 0.9732253086419753
Accuracy on train: 0.976440329218107
Accuracy on test: 0.9635802469135802
Gini
Accuracy on total: 0.9738425925925925
Accuracy on train: 0.9775720164609053
Accuracy on test: 0.9626543209876544


## b-2) Train: 75%, max_depth = 8

In [53]:
question(df, train_size=0.75, max_depth=8)

Entropy
Accuracy on total: 0.9929783950617284
Accuracy on train: 1.0
Accuracy on test: 0.9719135802469135
Gini
Accuracy on total: 0.9944444444444445
Accuracy on train: 1.0
Accuracy on test: 0.9777777777777777


## c-1) Train: 50%, max_depth = 6

In [54]:
question(df, train_size=0.5, max_depth=6)

Entropy
Accuracy on total: 0.9655092592592592
Accuracy on train: 0.9783950617283951
Accuracy on test: 0.9526234567901235
Gini
Accuracy on total: 0.9678240740740741
Accuracy on train: 0.9796296296296296
Accuracy on test: 0.9560185185185185


## c-2) Train: 50%, max_depth = 8

In [58]:
question(df, train_size=0.5, max_depth=8)

Entropy
Accuracy on total: 0.9790123456790123
Accuracy on train: 1.0
Accuracy on test: 0.9580246913580247
Gini
Accuracy on total: 0.9786265432098765
Accuracy on train: 1.0
Accuracy on test: 0.957253086419753
